# Implement Minimal Model Example
Test if model in README.md actually works

In [1]:
import xso
import numpy as np

@xso.component
class Variable:
    var = xso.variable(dims=[(), 'var'], description='basic state variable')
    
@xso.component
class LinearGrowth:
    var_ext = xso.variable(foreign=True, dims=[(), 'var'], flux='growth', description='external state variable')
    rate = xso.parameter(description='linear growth rate')
    @xso.flux(dims=[(), 'var'])
    def growth(self, var_ext, rate):
        return var_ext * rate + self.m.log(1)
    
model = xso.create({'Var':Variable,'Growth':LinearGrowth})

input_ds = xso.setup(solver='stepwise',
                     model=model,
                     time=np.arange(1,10),
                     input_vars={
                         'Var':{'var_label':'X', 'var_init':1},
                         'Growth':{'var_ext':'X', 'rate':1.},
                     })

with model:
    output_ds = input_ds.xsimlab.run()
    
print(output_ds)

input_ds = xso.setup(solver='odeint',
                     model=model,
                     time=np.arange(1,10),
                     input_vars={
                         'Var':{'var_label':'X', 'var_init':[1, 0.1]},
                         'Growth':{'var_ext':'X', 'rate':1.},
                     })

with model:
    output_ds = input_ds.xsimlab.run()
    

print(output_ds)

initializing model backend
Initializing Model Time
Within Solver, var_in_dict: defaultdict(None, {'time': 0.0})
Initializing component Var
Initializing component Growth
Within Solver, var_in_dict: defaultdict(None, {'time': 0.0, 'X': 1.0, 'Time_time_flux': 1.0})
assembling model
SOLVER : <xso.solvers.StepwiseSolver object at 0x7fdc43c5dc30>
Model is assembled:
Model contains: 
Variables:['time', 'X'] 
Parameters:['Growth_rate'] 
Forcings:[] 
Fluxes:['Time_time_flux', 'Growth_growth'] 
Full Model Dimensions:[('time', None), ('X', None), ('Time_time_flux', None), ('Growth_growth', None)] 

finalizing: cleanup
Model was solved in 0.00359 seconds
<xarray.Dataset>
Dimensions:               (time: 9, input_time: 9, clock: 9)
Coordinates:
  * clock                 (clock) int64 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: time, input_time
Data variables:
    Core__solver_type     <U8 'stepwise'
    Growth__growth_value  (time) float64 1.0 1.0 2.0 4.0 ... 32.0 64.0 128.0
    Growth__rate 

# DETAILED STEPS

In [1]:
import xso

import numpy as np

In [2]:
@xso.component
class Variable:
    var = xso.variable(description='basic state variable')

In [3]:
@xso.component
class LinearGrowth:
    var_ext = xso.variable(foreign=True, flux='growth', description='external state variable')
    rate = xso.parameter(description='linear growth rate')
    @xso.flux
    def growth(self, var_ext, rate):
        return var_ext * rate

In [4]:
model = xso.create({'Var':Variable,'Growth':LinearGrowth})

In [5]:
print(model)

<xsimlab.Model (5 processes, 6 inputs)>
Core
    solver_type     [in]
Time
    time            [in] ('input_time',) sequence of time points fo...
Var
    var_label       [in] label / basic state variable
    var_init        [in] initial value / basic state variable
Growth
    var_ext         [in] label reference / external state variable
    rate            [in] linear growth rate
Solver



In [6]:
input_ds = xso.setup(solver='stepwise',
                     model=model,
                     time=np.arange(1,10),
                     input_vars={
                         'Var':{'var_label':'X', 'var_init':1},
                         'Growth':{'var_ext':'X', 'rate':1.},
                     })

In [7]:
with model:
    output_ds = input_ds.xsimlab.run()

initializing model backend
Initializing Model Time
Within Solver, var_in_dict: defaultdict(None, {'time': 0.0})
Initializing component Var
Initializing component Growth
Within Solver, var_in_dict: defaultdict(None, {'time': 0.0, 'X': 1.0, 'Time_time_flux': 1.0})
finalizing: cleanup


TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [ ]:
output_ds